In [4]:
import pandas as pd
import dataprovider_pypots as dp
import process_edited as pce
import matplotlib.pyplot as plt
import torch

## Load Data

In [14]:
data = 'dataset/energy_data'
filename = f'{data}.csv'
real_df = pd.read_csv(filename)

real_df = real_df # You can load entire data if your memory is sufficient! 
real_df1 = real_df.drop(['date'], axis=1)
parser = pce.DataFrameParser().fit(real_df1, 1)
column_name = parser.column_name()
processed_data = torch.from_numpy(parser.transform())

data_dict = dp.TVMCG(real_df.iloc[0:2000,:], 
                     train_ratio=0.8, 
                     val_ratio=0.1, 
                     test_ratio=0.1, 
                     seq_len = 24, 
                     stride = 1, 
                     column_list=column_name, 
                     target_column=["lights", "rv1"])

KeyError: 'date'

In [12]:
data_dict

{'n_steps': 24,
 'n_features': 8,
 'train_X_ori': tensor([[[0.0000, 0.0000, 9.0000,  ..., 0.1693, 0.5362, 0.1537],
          [0.0000, 0.0000, 9.0000,  ..., 0.2999, 0.6709, 0.1565],
          [0.0000, 0.0000, 7.0000,  ..., 0.2317, 0.3914, 0.1904],
          ...,
          [1.0000, 0.0000, 8.0000,  ..., 0.3730, 0.2175, 0.2247],
          [0.0000, 1.0000, 0.0000,  ..., 0.2491, 0.2616, 0.0816],
          [0.0000, 0.0000, 0.0000,  ..., 0.1724, 0.3934, 0.1241]],
 
         [[0.0000, 0.0000, 9.0000,  ..., 0.2999, 0.6709, 0.1565],
          [0.0000, 0.0000, 7.0000,  ..., 0.2317, 0.3914, 0.1904],
          [1.0000, 0.0000, 5.0000,  ..., 0.3958, 0.4124, 0.3586],
          ...,
          [0.0000, 1.0000, 0.0000,  ..., 0.2491, 0.2616, 0.0816],
          [0.0000, 0.0000, 0.0000,  ..., 0.1724, 0.3934, 0.1241],
          [0.0000, 0.0000, 4.0000,  ..., 0.3262, 0.4841, 0.0691]],
 
         [[0.0000, 0.0000, 7.0000,  ..., 0.2317, 0.3914, 0.1904],
          [1.0000, 0.0000, 5.0000,  ..., 0.3958, 0.4124, 

## Train TimeAutoDiff

In [7]:
import VAE as vae
import DIFF as diff

VAE_training = 30000; diff_training = 30000; lat_dim = 6; 

############ Parameter Setting ############
# pip install ipywidgets
n_epochs = VAE_training; eps = 1e-5
weight_decay = 1e-6; lr = 2e-4; hidden_size = 512; num_layers = 2; batch_size = 100
channels = 64; min_beta = 1e-5; max_beta = 0.1; emb_dim = 128; time_dim = 8; threshold = 1; device = 'cuda'

############ Auto-encoder Training ############
ds = vae.train_autoencoder(real_df1, channels, hidden_size, num_layers, lr, weight_decay, n_epochs, batch_size, threshold, min_beta, max_beta, emb_dim, time_dim, lat_dim, device, data_dict)
latent_features = ds[1]

############ Diffusion Training ############
n_epochs = diff_training; hidden_dim = 200; num_layers = 2; diffusion_steps = 100; num_classes = len(latent_features)
diff = diff.train_diffusion(latent_features, real_df1, data_dict, hidden_dim, num_layers, diffusion_steps, n_epochs, num_classes)

Output()

/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [63,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [63,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [63,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [63,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [63,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [63,0,0], thread: [37,0,0] Assertion `srcIndex < srcSel

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Inference

In [ ]:
import numpy as np
import DIFF as Diff

target_mask_test=data_dict['target_mask_test']
target_test=data_dict['response_test']
cond_test=data_dict['cond_test']
time_info_test=data_dict['time_info_test']

# Sampling process
device = 'cuda'
diffusion_steps = 100
Batch_size, Seq_len, _ = target_test.shape
Lat_dim = lat_dim
t_grid = torch.linspace(0, 1, Seq_len).view(1, -1, 1).to(device)
samples = Diff.sample(
    t_grid.repeat(Batch_size, 1, 1),
    Batch_size,
    Seq_len,
    Lat_dim,
    diffusion_steps,
    diff,
    time_info_test,
    cond_test,
    target_mask_test.float().to(device),
)

# Process the generated data
gen_output = ds[0].decoder(samples.to(device), target_mask_test, cond_test)
_synth_data = pce.convert_to_tensor(real_df1, gen_output, 1, Batch_size, Seq_len)

## Draw the predicted column

In [ ]:
label = 1 # 10/6/2012 - Saturday
##################################################################################################################
# plot label's time series snapshot
target_column = ['weather_main','temp']
fig, axes = plt.subplots(1, len(target_column), figsize=(20, 4)) 

_synth_data = _synth_data[:,:,[3,5]]
target_test = target_test[:,:,[3,5]]

# Plot histogram for the specified column
for i in range(0,len(target_column)):
    axes[i].plot(target_test[label,:,i],  marker='o', linestyle='-', color='b')
    axes[i].plot(_synth_data[label,:,i].cpu(),  marker='o', linestyle='-', color='r')
    axes[i].set_xlabel('Time')
    axes[i].set_ylabel(f'{column_name[i]}')
    axes[i].grid(True)
    axes[i].legend(loc='upper left')

plt.show()
